# Содержание

1. [Обзор данных](#start)
2. [Предобработка данных](#pre)   
    2.1. [Обработка пропусков](#pre_null)    
    2.2. [Замена типов данных](#pre_type)   
    2.3. [Обработка дубликатов](#pre_duplicates)   
    2.4. [Лемматизация](#pre_lemmos)   
    2.5. [Категоризация данных](#pre_category)   
3. [Проверка гипотез](#hypotis)   
    3.1. [Есть ли зависимость между наличием детей и возвратом кредита в срок?](#hyp1)   
    3.2. [ Есть ли зависимость между семейным положением и возвратом кредита в срок?](#hyp2)   
    3.3. [Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#hyp3)   
    3.4. [Как разные цели кредита влияют на его возврат в срок?](#hyp4)
4. [Общий вывод](#result)


# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** - проверить несколько гипотез:
- Количество детей заемщика влияет на погашение кредита. Большое количество детей увеличивает риск просрочки платежей.
- Семейные люди чаще гасят кредит в срок по сравнению с холостыми. Они более платежеспособные.
- Уровень дохода влияет на погашение кредита. Чем он выше, тем чаще нет задолженностей.
- Какие цели кредита менее рискованные с точки зрения возврата кредита.

##  Обзор данных
<a id='start'></a>

In [1]:
!pip install pymystem3==0.2.0 -U

In [2]:
import pandas as pd # импорт бибилиотеки pandas 

from pymystem3 import Mystem # импортируем из библиотеки pymystem3 функцию Mystem()
m = Mystem() # сохраняем функцию Mystem() в переменную

from collections import Counter # импортируем контейнер Counter из модуля collections

In [3]:
df = pd.read_csv('data.csv') # чтение исходного файла и сохранение его в переменную df
df.head(10) # вывод первых 10 строк таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Дополнительно получаем общую информацию о таблице:

In [4]:
df.info() # вывод общей информации о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Таблица содержит 12 столбцов:
- `children` — количество детей в семье;
- `days_employed` — общий трудовой стаж в днях;
- `dob_years` — возраст клиента в годах;
- `education` — уровень образования клиента;
- `education_id` — идентификатор уровня образования;
- `family_status` — семейное положение;
- `family_status_id` — идентификатор семейного положения;
- `gender` — пол клиента;
- `income_type` — тип занятости;
- `debt` — имел ли задолженность по возврату кредитов;
- `total_income` — ежемесячный доход;
- `purpose`— цель получения кредита.

В названиях столбщов все корректно. 

В данных видны проблемы со значениями:
1. Общий трудовой стаж имеет отрицательные значения, указан в днях (что неудобно для анализа) и имеет тип данных float.
2. Уровень образования клиентов типа object и имеет разный способ написания

Количество значений в столбцах `days_employed` и `total_income` однинаково и ниже, чем в остальных. Значит есть пропущенные значения и, возможно, они взаимосвязаны.

В таблице указаны общие данные о заемщиках - их уровень образования, возраст, пол, семейное положение и количество детей. Также есть информация о рабочем стаже, доходе, целях кредитования. Для каждого заемщика известно, имел ли он ранее задолженности по возврату кредита.

Для того, чтобы двигаться дальше, необходимо проверить данные на пропуски, удалить повторения и привести ряд значений к удобному для анализа виду. 

## Предобработка данных
<a id='pre'></a>

### Обработка пропусков
<a id='pre_null'></a>

Сначала посчитаем пропущенные значения в таблице:

In [5]:
df.isna().sum() # считаем количество пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Проверим, есть ли связь между отсутствием данных и определенной занятостью потребителей:

In [6]:
# группируем строки с отсутствующими данными по типу занятости
df[df['total_income'].isna() & df['days_employed'].isna()].groupby('income_type')['income_type'].count()

income_type
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
Name: income_type, dtype: int64

**Выводы**

Отсутствие данных по стажу и доходу в одних и тех же строках.

Это отсутствие не привязано к конкретной занятости. Скорее всего, это техническая ошибка. 

### Замена типа данных
<a id='pre_type'></a>

Строки с отсутствующими данными занимают около 10% и содержат другую ключевую информацию для исследования.

Поэтому отсутствующие данные по стажу и доходу заменим на медианные значения для минимизации влияния выбросов.

Перед тем, как заполнять пропуски, проверим все столбцы на корректность существущих значений, в том числе:
- Возраст заемщиков `dob_years` - от 18 лет
- В столбце `days_employed` количество лет стажа иногда превышает возраст заемщика, а также имеет отрицательные значения;
- Значения в столбце `education` написаны в разном стиле написания
- Цели кредита `purpose` имеют разное написание, но много одинаковых по смыслу.

In [7]:
df.loc[df['dob_years'] < 18, 'dob_years'].value_counts() # выведем тех, кому меньше 18

0    101
Name: dob_years, dtype: int64

В таблице заемщики младше 18 - только с нулевым значением. Проверим, связано ли это с типом занятости.

In [8]:
# выведем тип занятости income_type тех, у кого возраст dob_years 0
df[df['dob_years'] == 0].groupby('income_type')['income_type'].count()

income_type
госслужащий     6
компаньон      20
пенсионер      20
сотрудник      55
Name: income_type, dtype: int64

Не прослеживается зависимости нулевого возраста с типом занятости. Скорее всего, это ошибка ввода или техническая ошибка. Таких значений менее 0,5%, поэтому удалим их.

In [9]:
df.drop(df[df['dob_years'] == 0].index, inplace=True) # удаляем строки, где возраст dob_years равен нулю
df = df.reset_index(drop=True) # обновляем индексацию

Проверим также стаж `days_employed` и доход `total_income` на нулевые значения.

In [10]:
df.loc[df['days_employed'] == 0, 'income_type'].count() # выведем нулевые значения days_employed

0

In [11]:
df.loc[df['total_income'] == 0, 'income_type'].count() # выведем нулевые значения total_income

0

Нулевых значений нет. Поэтому пока для дальнейших математических расчетов, заменим пропуски нулями и преобразуем значения из вещественного типа в целочисленный.

In [12]:
# заменяем пропуски в столбцах days_employed и total_income на нули
df[['days_employed','total_income']] = df[['days_employed','total_income']].fillna(0)
# меняем тип данных в столбцах days_employed и total_income на целочисленный
df[['days_employed','total_income']] = df[['days_employed','total_income']].astype('int')
df.info() # проверяем отсутствие пропусков и тип данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21424 entries, 0 to 21423
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21424 non-null  int64 
 1   days_employed     21424 non-null  int32 
 2   dob_years         21424 non-null  int64 
 3   education         21424 non-null  object
 4   education_id      21424 non-null  int64 
 5   family_status     21424 non-null  object
 6   family_status_id  21424 non-null  int64 
 7   gender            21424 non-null  object
 8   income_type       21424 non-null  object
 9   debt              21424 non-null  int64 
 10  total_income      21424 non-null  int32 
 11  purpose           21424 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


Теперь вернемся к самим значениям таблицы. В столбце `days_employed` количество лет стажа иногда превышает возраст заемщика, а также иногда имеет отрицательные значения.

In [13]:
df['days_employed'] = abs(df['days_employed']) # заменим отрицательные значения days_employed на положительные

In [14]:
# выведем те типы занятости income_type, для кого стаж days_employed превышает возраст dob_years
df.loc[df['days_employed'] >= df['dob_years']*365,'income_type'].value_counts()

пенсионер      3426
безработный       2
Name: income_type, dtype: int64

Аномально высокое значение стажа встречается преимущестенно у пенсионеров. Посмотрим, сколько всего пенсионеров в таблице и что со стажем у остальных.

In [15]:
df.loc[df['income_type'] == 'пенсионер', 'income_type'].count() # сколько всего пенсионеров income_type

3836

In [16]:
# посчитаем среднее значение стажа days_employed для пенсионеров income_type, 
# чей возраст dob_years больше стажа days_employed
df.loc[(df['income_type'] == 'пенсионер') & (df['days_employed'] < df['dob_years']*365 ), 'days_employed'].mean()

0.0

У большинства пенсионеров аномально высокое значение стажа, у остальных ноль (т.е. отсутствуют значения). Скорее всего это ошибка при вводе связана с единицами измерения. Возможно дни посчитаны в часах. Проверим это по значениям стажа во всей таблице. Далее исправим это значение у тех, где оно есть, а отсутсвующие заменим на медианное по всем пенсионерам.

In [17]:
print('Аномальный стаж -', round(df.loc[df['days_employed'] > df['dob_years']*365,'days_employed'].mean()/365), 'лет.')
print('Максимальный адекватный стаж', round(df.loc[df['days_employed'] < df['dob_years']*365,'days_employed'].max()/365), 'лет.')
print('Средний адекватный стаж', round(df.loc[df['days_employed'] < df['dob_years']*365,'days_employed'].mean()/365), 'лет.')

Аномальный стаж - 1000 лет.
Максимальный адекватный стаж 50 лет.
Средний адекватный стаж 6 лет.


In [18]:
df.info() # выведм информацю об измененной таблице 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21424 entries, 0 to 21423
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21424 non-null  int64 
 1   days_employed     21424 non-null  int32 
 2   dob_years         21424 non-null  int64 
 3   education         21424 non-null  object
 4   education_id      21424 non-null  int64 
 5   family_status     21424 non-null  object
 6   family_status_id  21424 non-null  int64 
 7   gender            21424 non-null  object
 8   income_type       21424 non-null  object
 9   debt              21424 non-null  int64 
 10  total_income      21424 non-null  int32 
 11  purpose           21424 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


In [19]:
def days_employed_replace(row):     # создадим функцию по замене значения стажа 
    days = row['days_employed']     # стаж days возьмем из столбца days_employed
    age = row['dob_years']          # возраст age возьмем из столбца dob_years
    if days/365 >= age:             # если стаж days больше возраста age
        days_replace = days / 24                   # делим стаж days на 24
        return days_replace                 # возвращаем стаж days
    return days
df['days_employed'] = df.apply(days_employed_replace, axis=1) # применим функцию days_employed_replace
df.loc[df['days_employed'] >= df['dob_years']*365, 'income_type'].count() # проверяем результаты функции

54

In [20]:
df.info() # выведм информацю об измененной таблице 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21424 entries, 0 to 21423
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21424 non-null  int64  
 1   days_employed     21424 non-null  float64
 2   dob_years         21424 non-null  int64  
 3   education         21424 non-null  object 
 4   education_id      21424 non-null  int64  
 5   family_status     21424 non-null  object 
 6   family_status_id  21424 non-null  int64  
 7   gender            21424 non-null  object 
 8   income_type       21424 non-null  object 
 9   debt              21424 non-null  int64  
 10  total_income      21424 non-null  int32  
 11  purpose           21424 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(5)
memory usage: 1.9+ MB


In [21]:
df[df['days_employed'] >= df['dob_years']*365] # выводим данные об оставшихся

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
155,0,14517.250000,38,среднее,1,женат / замужем,0,F,пенсионер,1,113560,сделка с автомобилем
747,0,16281.458333,41,среднее,1,женат / замужем,0,M,пенсионер,0,151898,операции со своей недвижимостью
772,0,15222.333333,38,среднее,1,женат / замужем,0,F,пенсионер,0,73859,покупка недвижимости
1235,0,13948.500000,22,Среднее,1,Не женат / не замужем,4,F,пенсионер,0,89368,получение высшего образования
1376,0,14741.750000,37,среднее,1,вдовец / вдова,2,F,пенсионер,0,216452,строительство недвижимости
1438,0,14122.458333,37,СРЕДНЕЕ,1,гражданский брак,1,M,пенсионер,0,148657,свадьба
1629,0,16635.833333,39,среднее,1,женат / замужем,0,M,пенсионер,0,211513,операции с недвижимостью
1744,1,16249.125000,38,ВЫСШЕЕ,0,Не женат / не замужем,4,F,пенсионер,0,311574,покупка недвижимости
2976,0,15114.208333,39,среднее,1,в разводе,3,F,пенсионер,0,96891,покупка недвижимости
3116,1,14063.500000,31,среднее,1,женат / замужем,0,M,безработный,1,59956,покупка жилья для сдачи


В таблице мы видим, что в 54 строках не соответствуют друг другу сразу три столбца - `days_employed`, `dob_years` `income_type`. Стаж все еще (после уменьшения в 24 раза!) превышает возраст, а возраст существенно ниже пенсионного. Цели кредита ясности тоже не вносят. Эти строки лучше удалить (включая строки с двумя безработными).

In [22]:
# удаляем строки, где стаж days_employed все еще больше возраста dob_years
df.drop(df[df['days_employed'] >= df['dob_years']*365].index, inplace=True)
df = df.reset_index(drop=True) # обновляем индексацию

Теперь восполним нулевые (пропущенные) значения стажа `days_employed` и дохода `total_income` медианными значениями в соответствии с возрастом `dob_years` и типом занятости `income_type` соответственно.

In [23]:
for dob_years in df['dob_years'].unique():
    median = df.loc[(df['days_employed'] != 0) & (df['dob_years'] == dob_years), 'days_employed'].median()
    df.loc[(df['days_employed'] == 0) & (df['dob_years'] == dob_years), 'days_employed'] = median

In [24]:
for income_type in df['income_type'].unique():
    median = df.loc[(df['total_income'] != 0) & (df['income_type'] == income_type), 'total_income'].median()
    df.loc[(df['total_income'] == 0) & (df['income_type'] == income_type), 'total_income'] = median

Проверим типы данных в таблице:

Для удобства анализа данных изменим тип данных дохода в столбце `total_income` и `days_employed` методом `astype()` - с целыми числами работать удобнее.

In [25]:
# изменим тип данных total_income и days_employed на целочисленный
df[['days_employed','total_income']] = df[['days_employed','total_income']].astype('int') 

Заменим значения в столбце `education` на однотипные. А также проверим `education` и `education_id` на соответствие.

In [26]:
df['education'] = df['education'].str.lower() # сделаем написание значений в столбце education строчными буквами
df['education'].value_counts() # выводим все возможные значения с столбце education

среднее                15127
высшее                  5215
неоконченное высшее      742
начальное                280
ученая степень             6
Name: education, dtype: int64

In [27]:
df['education_id'].value_counts() # выведем уникальные значения в столбце education_id

1    15127
0     5215
2      742
3      280
4        6
Name: education_id, dtype: int64

Значения в столбцах `education` и `education_id` соответствуют соответствуют друг другу.

Проверим типы написания в столбцах `family_status` и `family_status_id`, и их соответствие.

In [28]:
df['family_status'].value_counts() # выведем уникальные значения в столбце family_status

женат / замужем          12311
гражданский брак          4136
Не женат / не замужем     2789
в разводе                 1180
вдовец / вдова             954
Name: family_status, dtype: int64

In [29]:
df['family_status_id'].value_counts() # выведем уникальные значения в столбце family_status_id

0    12311
1     4136
4     2789
3     1180
2      954
Name: family_status_id, dtype: int64

В `family_status` и `family_status_id` все в порядке.

Теперь пройдемся по столбцу `children`:

In [30]:
df['children'].value_counts() # выведем уникальные значения в столбце children

 0     14042
 1      4792
 2      2038
 3       327
 20       75
-1        46
 4        41
 5         9
Name: children, dtype: int64

Предположим, что значения -1 это опечатка и значение соответствует 1. А значение 20 тоже опечатка и соответствует 2.
Заменим указанные значения:

In [31]:
df.loc[df['children'] == 20, 'children'] = 2 # заменим значения 20 на 2
df.loc[df['children'] == -1, 'children'] = 1 # заменим значения -1 на 1

Теперь проверим столбец `income_type` на повторы:

In [32]:
df['income_type'].value_counts() # выведем уникальные значения income_type

сотрудник          11064
компаньон           5065
пенсионер           3784
госслужащий         1453
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Теперь проверим столбец `purpose` на повторы:

In [33]:
df['purpose'].value_counts() # выведем уникальные значения purpose

свадьба                                   786
на проведение свадьбы                     770
сыграть свадьбу                           765
операции с недвижимостью                  672
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   648
операции с жильем                         646
операции с коммерческой недвижимостью     645
покупка жилья                             641
покупка жилья для семьи                   640
жилье                                     638
строительство собственной недвижимости    633
операции со своей недвижимостью           629
недвижимость                              629
строительство жилой недвижимости          622
строительство недвижимости                619
покупка своего жилья                      618
покупка недвижимости                      617
ремонт жилью                              609
покупка жилой недвижимости                604
на покупку своего автомобиля              502
автомобиль                        

**Выводы**

Пропущенные данные дополнены. Написание в толбце `education` приведены к единому стилю. Найдены следующие артефакты в некторых строках:
- Стаж превышает возраст;
- Возраст заемщиков равен нулю;
- Столбец `purpose` нуждается в категоризации.

### Обработка дубликатов
<a id='pre_duplicates'></a>

Будем искать явные дубликаты:

In [34]:
df.duplicated().sum() # узнаем количество явных повторов

71

In [35]:
df = df.drop_duplicates().reset_index(drop=True) # удаляем явные дубликаты, заново индексируем строки

**Выводы**

Явные дубликаты удалены. Количество дублекатов увеличилось после устранения разночтений в столбцах. Все это говорит об ошибках при вводе данных, а также о вероятной техничекой ошибке при формировании архива. Других неявных дубликатов найти нет возможности в связи с отсутствием id заемщика (или аналогичного идентификатора).

### Лемматизация
<a id='pre_lemmos'></a>

Для дальнейшего анализа необходимо выполнить лемматизацию значений столбца `purpose`, для дальнейшей катерогизации.

In [36]:
Counter(m.lemmatize(' '.join(df['purpose'].tolist()))) # посчитаем леммы целей кредита purpose

Counter({'покупка': 5861,
         ' ': 54660,
         'жилье': 4427,
         'приобретение': 459,
         'автомобиль': 4278,
         'дополнительный': 900,
         'образование': 3982,
         'сыграть': 756,
         'свадьба': 2297,
         'операция': 2586,
         'с': 2896,
         'на': 2206,
         'проведение': 761,
         'для': 1283,
         'семья': 637,
         'недвижимость': 6315,
         'коммерческий': 1302,
         'жилой': 1223,
         'строительство': 1871,
         'собственный': 633,
         'подержать': 850,
         'свой': 2221,
         'со': 626,
         'заниматься': 896,
         'сделка': 936,
         'получение': 1302,
         'высокий': 1360,
         'подержанный': 112,
         'профильный': 435,
         'сдача': 646,
         'ремонт': 604,
         '\n': 1})

По смыслу можно выделить следующие ключевые цели:
- покупка/строительство жилья (недвижимости);
- покупка автомобиля;
- оплата образования;
- и другое.

In [37]:
# добавим в таблицу столбец с назначениями целей
new_txt=[]
text_total=''
for text_part in df['purpose']:
    text_total += (text_part + ' br ')
lemmos = m.lemmatize(text_total)    
for word in lemmos:
    if word != 'br' and word.strip() != '':
        if word == 'свадьба':
            new_txt.append(word)
        elif word == 'образование':
            new_txt.append(word)
        elif word == 'автомобиль':
            new_txt.append(word)
        elif (word == 'жилье') | (word == 'недвижимость'):
            new_txt.append('недвижимость')
            
purpose_group = pd.Series(data=new_txt)
df['purpose_group'] = purpose_group

df['purpose_group'].value_counts() # выведем уникальные новых категорий purpose

недвижимость    10742
автомобиль       4278
образование      3982
свадьба          2297
Name: purpose_group, dtype: int64

**Выводы**

При внимательном рассмотрении целей займов, их можно объединить в четыре группы:
- покупка недвижимости;
- покупка автомобиля;
- оплата образования;
- организация свадьбы.

Покупка недвижимости - самый востребованный вид займа. Примерно половина от всех займов.

### Категоризация данных
<a id='pre_category'></a>

Заемщиков с детьми разделим на три категории:
- без детей
- с детьми (1-2 ребенка)
- многодетные (от 3х детей)

In [38]:
def children_group(data):         # создаем функцию children_group для разделения заемщиков на три группы
    children = data['children']   # значение для группировки по количеству детей в столбце children
    if children >= 3:             # если не менее трех
        return 'многодетный'      # возвращаем многодетный
    elif children > 0:            # если 1-2 
        return 'с детьми'         # возвращаем с детьми
    else:                         # если ноль
        return 'без детей'        # возвращаем без детей

df['children_group'] = df.apply(children_group, axis=1) # применяем функцию children_group к таблице
df['children_group'].value_counts() # выведем уникальные новых категорий children

без детей      13984
с детьми        6938
многодетный      377
Name: children_group, dtype: int64

Основные заемщики - люди, не имеющие детей. Их примерно 2/3. Совсем редко кредиты берут многодетные заемщики - менее 2%.

Теперь рассмотрим заемщиков по семейному положению. Для этого разделим всех заемщиков на две группы:
- семейные
- холостые

In [39]:
def family_group(data):                             # создаем функцию family_group для разделения заемщиков на две группы
    family_status = data['family_status']           # значение для группировки из столбца family_status
    if ('женат / замужем' in family_status or       # если статус женат/замужем или
        'гражданский брак' in family_status):       # гражданский брак
        return 'семейный'                           # возвращаем семейный
    else:                                           # если другое
        return 'холостой'                           # возвращаем холостой

df['family_group'] = df.apply(family_group, axis=1) # применяем функцию family_group к таблице

df['family_group'].value_counts() # выведем уникальные новых категорий family_status

семейный    16380
холостой     4919
Name: family_group, dtype: int64

В целом среди всех заемщиков семейных больше холостых более чем в три раза.

Далее будем рассматривать заемщиков по уровню дохода. Сначала определим минимальное и максимальное значения и сравним их со средним значением.

In [40]:
print('Минимальный ежемесячный доход:', df['total_income'].min(), 'рублей')  # минимальный доход
print('Максимальный ежемесячный доход:', df['total_income'].max(), 'рублей') # максимальный доход
print('Средний ежемесячный доход', df['total_income'].median(), 'рублей') # средний доход (медианное значение)

Минимальный ежемесячный доход: 20667 рублей
Максимальный ежемесячный доход: 2265604 рублей
Средний ежемесячный доход 142594.0 рублей


Судить о корректности данных по доходам сложно, поскольку не ясно, как эти доходы указывались - со слов заемщиков или подтверждались документально.

Разделим заемщиков по доходу на следущие группы:
- ниже среднего 
- выше среднего 

In [41]:
def total_income_group(data):                   # создаем функцию total_income_group для разделения заемщиков на четыре группы
    total_income = data['total_income']         # значение для группировки по доходу в столбце total_income
    if total_income < 150000:                   # если доход до 150 000
        return 'ниже среднего'                  # возвращаем ниже среднего
    else:                                       # если выше 150 000
        return 'выше среднего'                  # возвращаем выше среднего
    

df['total_income_group'] = df.apply(total_income_group, axis=1) # применяем функцию total_income_group к таблице
df['total_income_group'].value_counts() # выведем уникальные новых категорий total_income

ниже среднего    11535
выше среднего     9764
Name: total_income_group, dtype: int64

Разделили заемщиков на две группы отномительно среднего дохода. Их количество сопоставимо.

**Вывод**

Категории выделены. Теперь можно переходить к проверке гипотез.

## Проверка гипотез
<a id='hypotis'></a>

### Есть ли зависимость между наличием детей и возвратом кредита в срок?
<a id='hyp1'></a>

In [42]:
format_per='{:.1%}' # задаем формат вывода значений в процентах
# процент невозврата кредита в зависимости от количества детей
df.groupby('children').agg({'debt': [lambda x: format_per.format(x.mean()), 'count', 'sum']}).reset_index()

children       debt             
           <lambda_0>  count   sum
0        0       7.5%  13984  1054
1        1       9.1%   4828   439
2        2       9.5%   2110   201
3        3       8.3%    327    27
4        4       9.8%     41     4
5        5       0.0%      9     0

In [43]:
# процент возврата кредита разными категориями заемщиков по наличию детей
df.groupby('children_group').agg({'debt': [lambda x: format_per.format(x.mean()), 'count', 'sum']}).reset_index()

children_group       debt             
                 <lambda_0>  count   sum
0      без детей       7.5%  13984  1054
1    многодетный       8.2%    377    31
2       с детьми       9.2%   6938   640

**Выводы**

1. Заемщики, не имеющие детей, реже имеют задолженность по кредитам. Среди заемщиков их существенно больше.
2. Многодетные заемщики чаще выплачивают кредит, чем заемщики с одним или двумя детьми. Однако, их очень немного в выборке.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?
<a id='hyp2'></a>

In [44]:
# среднее значение возвратов кредита с разным семейным статусом
df.groupby('family_status').agg({'debt': [lambda x: format_per.format(x.mean()), 'count', 'sum']}).reset_index()

family_status       debt            
                         <lambda_0>  count  sum
0  Не женат / не замужем       9.8%   2786  272
1              в разводе       7.2%   1180   85
2         вдовец / вдова       6.5%    953   62
3       гражданский брак       9.3%   4110  383
4        женат / замужем       7.5%  12270  923

In [45]:
# среднее значение возвратов кредита для семейных и холостых
df.groupby('family_group').agg({'debt': [lambda x: format_per.format(x.mean()), 'count', 'sum']}).reset_index()

family_group       debt             
               <lambda_0>  count   sum
0     семейный       8.0%  16380  1306
1     холостой       8.5%   4919   419

**Выводы**

1. В целом семейные заемщики возвращают кредит чаще, чем холостые - примерно на 0,5%. Семейных заемщиков в разы больше холостых.
2. При этом самые надежные заемщики - вдовы и вдовцы, а самые ненадежные - не женатые и не замужние.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
<a id='hyp3'></a>

In [46]:
# среднее значение возвратов кредита в зависимости от уровня дохода
df.groupby('total_income_group').agg({'debt': [lambda x: format_per.format(x.mean()), 'count', 'sum']}).reset_index()

total_income_group       debt            
                     <lambda_0>  count  sum
0      выше среднего       7.8%   9764  758
1      ниже среднего       8.4%  11535  967

**Выводы**

1. Заемщики, имеющие доход ниже среднего, хуже возвращают кредит в срок.
2. Люди с доходом ниже среднего берут кредит чаще. Или, возможно, их просто больше.

### Как разные цели кредита влияют на его возврат в срок?
<a id='hyp4'></a>

In [47]:
# среднее значение возвратов кредита в зависимости от цели кредита
df.groupby('purpose_group').agg({'debt': [lambda x: format_per.format(x.mean()), 'count', 'sum']}).reset_index()

purpose_group       debt            
                <lambda_0>  count  sum
0    автомобиль       9.3%   4278  398
1  недвижимость       7.2%  10742  778
2   образование       9.2%   3982  367
3       свадьба       7.9%   2297  182

**Выводы**

1. Возвраты кредитов в срок, взятые с целью покупки недвижимости, чаще других. При том, что эта цель самая популярная.
2. Больше всего задолженностей по кредитам, взятым на покупку автомобиля и оплату образования.

## Общий вывод
<a id='result'></a>

Мы проверили четыре гипотезы:

1. Наличие детей влияет на возврат кредита в срок. 

Первая гипотеза подвтвердилась частично - заемщики без детей имеют задолженность по кредиту реже. Но судить о том, что увеличение детей увеличивает количество задолженностей не можем, т.к. заемщиков с детьми больше 2х мало.

2. Семейные заемщики чаще возвращают кредит в срок.

Вторая гипотеза подтвердилась полностью. Семейные заемщики чаще холостых возвращают кредит в срок. Интересно, что овдовевшие заемщики возвращают кредит существенно лучше не женатых заемщиков.

3. Возврат кредита в срок связан с уровенем ежемесячного дохода.

Третья гипотеза подтвердилась полностью. Заемщики с более низким доходом возвращают кредит хуже.

4. Влияют ли цели кредита на срок его возврата.

Гипотеза подтвердилась - кредиты, взятые с целью покупки недвижимости, возвращаются в срок лучше других. Кредиты на приобритение автомобиля или оплату образования чаще имеют просрочку. 

**В целом общий процент невозврата кредита заемщиками не превышает 10%**